In [1]:
!pwd

/home/amaro/notebooks


# Extract and reconstruction

In [2]:
#!git clone -b v7 https://github.com/CYGNUS-RD/reconstruction.git  
#!wget -P ./reconstruction/pedestals/ -N http://191.252.179.152/files/pedmap_run2155_rebin1.root
#!wget -P ./reconstruction/ -N http://191.252.179.152/files/configFilef55.txt
#!wget -P /mnt/ssdcache/amaro/ -N https://s3.cloud.infn.it/v1/AUTH_2ebf769785574195bde2ff418deac08a/cygnus/Data/LAB/histograms_Run02163.root
#!mv /content/reconstruction/pedestals/pedmap_run02155_rebin1.root /content/reconstruction/pedestals/pedmap_run2155_rebin1.root
#!mkdir -p ./plots
#!sed -i 's/expand_noncore, debug=True/expand_noncore, debug=False/g' ./reconstruction/cluster/ddbscan_inner.py

# Options

In [3]:
import numpy as np
from pathlib import Path

tries = 1
ini = 0
fin = 0
sizes = [2, 4, 8, 16, 32]
#sizes = [4]
# sizes = [4, 2]
# sizes = [1, 2, 3, 4, 5, \
#         6, 7, 8, 9, 10,\
#         11, 12, 13, 14,\
#         15, 16, 17, 18,\
#         19, 20, 21, 22,\
#         23, 24, 25, 26,\
#         27, 28, 29, 30,\
#         31, 32]

t_cython = np.zeros([tries,(fin-ini+1)]) 
t_nocyth = np.zeros([tries,(fin-ini+1)]) 
numberofevents = [1000]#[10, 25, 50, 100, 150, 200]

path = Path(r'/home/amaro/notebooks/simulations')
#path = Path(r'/home/amaro/notebooks/aquisitions') #done
#path = Path(r'/home/amaro/notebooks/pedestals') #done

# Cython module

In [4]:
!pwd

/home/amaro/notebooks


In [5]:
%load_ext Cython

In [6]:
%%cython
import numpy as np
cimport numpy as np

DTYPE = np.float

ctypedef np.float_t DTYPE_t


def nred_cython(np.ndarray[DTYPE_t, ndim=2] edges, int escala, float meancut=0.35):
    cdef int rescale = escala 
    cdef int tpx = 10
    cdef float mpx
    cdef float a,b,c,d,spx,f,g,h,i
    cdef int neighbors
    
    cdef int k, j

    for k in range(rescale):
        for j in range(2):
            edges[j,k]=0
            edges[k,j]=0
            edges[rescale-1-j,k]=0
            edges[k,rescale-1-j]=0

    for k in range(1,rescale-2):
        for j in range(1,rescale-2):
            a = edges[k-1,j-1]
            b = edges[k,j-1]
            c = edges[k+1,j-1]
            d = edges[k-1,j]
            spx = edges[k,j]
            f = edges[k+1,j]
            g = edges[k-1,j+1]
            h = edges[k,j+1]
            i = edges[k+1,j+1]
            mpx = (a+b+c+d+f+g+h+i)/8.
            # put very noisy pixels at the average value of the frame around
            if abs(spx - mpx) > tpx :
                edges[k,j] = mpx
            # filter the pixels with no sufficient energy around
            if (mpx < 0.35):
                edges[k,j] = 0
            # require at least two neighbors above threshold
            #frame = edges[k-1:k+2,j-1:j+2]
            neighbors = 9-(not a)-(not b)-(not c)-(not d)-(not spx)-(not f)-(not g)-(not h)-(not i)
            if neighbors<3:
                edges[k,j] = 0
    return edges


def sim3d_cython(np.ndarray[np.int_t, ndim=2] img_rb_zs, np.ndarray[np.int_t, ndim=2] points):
    cdef int size = points.shape[0]
    cdef int k, nreplicas=0
    cdef np.int_t j,l,idx1=0,idx2=0
    for k in range(size):
        j = points[k,0]
        l = points[k,1]
        if (img_rb_zs[j,l] == 0):
            nreplicas += 1
        else:
            nreplicas += img_rb_zs[j,l] 
    cdef np.ndarray[np.int64_t, ndim=2] newpoints = np.zeros((nreplicas,2), dtype=np.int64)
    cdef int count=0
    for k in range(size):
        j = points[k,0]
        l = points[k,1]
        nreplicas = img_rb_zs[j,l]
        if (nreplicas < 1):
            newpoints[idx1,0] = j
            newpoints[idx1,1] = l
            idx1 += 1
        else:
            newpoints[(idx1),0] = j
            newpoints[(idx1),1] = l
            idx1 += 1
            for count in range(nreplicas-1):
                newpoints[(idx2+size),0] = j
                newpoints[(idx2+size),1] = l
                idx2 += 1
    return newpoints


# With Cython

## Snakes module with cython

In [7]:
%cd reconstruction/

import numpy as np
import ROOT,math,os,sys,time
import pickle

from scipy.ndimage import gaussian_filter, median_filter
from skimage import img_as_float
from skimage.morphology import reconstruction
from skimage import measure

from morphsnakes import(morphological_chan_vese,
                        morphological_geodesic_active_contour,
                        inverse_gaussian_gradient,
                        checkerboard_level_set)

from clusterTools import Cluster
from cameraChannel import cameraTools
from cluster.ddbscan_ import DDBSCAN
from energyCalibrator import EnergyCalibrator

import debug_code.tools_lib as tl

class SnakesFactory:
    def __init__(self,img,img_fr,img_fr_zs,img_ori,vignette,name,options,geometry):
        self.name = name
        self.options = options
        self.rebin = options.rebin
        self.geometry = geometry
        self.ct = cameraTools(geometry)
        self.image = img
        self.img_ori = img_ori
        self.imagelog = np.zeros((self.image.shape[0],self.image.shape[1]))
        for (x,y),value in np.ndenumerate(self.image):
            if value > 3.0/math.sqrt(self.rebin): # tresholding needed for tracking
                self.imagelog[x,y] = math.log(value)
        self.image_fr    = img_fr
        self.image_fr_zs = img_fr_zs
        self.vignette = vignette
        self.contours = []
        
    def getClusters(self,plot=False):

        from sklearn.cluster import DBSCAN
        from sklearn import metrics
        from scipy.spatial import distance
        from scipy.stats import pearsonr
        from random import random

        outname = self.options.plotDir
        if outname and not os.path.exists(outname):
            os.system("mkdir -p "+outname)
            os.system("cp utils/index.php "+outname)
        
        #   Plot parameters  #
        
        vmin=1
        vmax=5
        
        tip = self.options.tip
        
        #-----Pre-Processing----------------#
        rescale=int(self.geometry.npixx/self.rebin)

        t0 = time.perf_counter()
        filtimage = median_filter(self.image_fr_zs, size=2)
        edges = self.ct.arrrebin(filtimage,self.rebin)
        edcopy = edges.copy()
        #edcopyTight = tl.noisereductor(edcopy,rescale,self.options.min_neighbors_average)
        edcopyTight = nred_cython(edcopy,rescale,self.options.min_neighbors_average)


        # make the clustering with DBSCAN algo
        # this kills all macrobins with N photons < 1
        points = np.array(np.nonzero(np.round(edcopyTight))).astype(int).T
        lp = points.shape[0]

        ## apply vignetting (if not applied, vignette map is all ones)
        ## this is done only for energy calculation, not for clustering (would make it crazy)
        image_fr_vignetted = self.ct.vignette_corr(self.image_fr,self.vignette)
        image_fr_zs_vignetted = self.ct.vignette_corr(self.image_fr_zs,self.vignette)
                
        if tip=='3D':
            sample_weight = np.take(self.image, self.image.shape[0]*points[:,0]+points[:,1]).astype(int)
            sample_weight[sample_weight==0] = 1
            X = points.copy()
            
        else:
            X = points.copy()
            sample_weight = np.full(X.shape[0], 1, dtype=np.int)

        # returned collections
        superclusters = []

        # clustering will crash if the vector of pixels is empty (it may happen after the zero-suppression + noise filtering)
        if len(X)==0:
            return superclusters

        # - - - - - - - - - - - - - -
        t1 = time.perf_counter()
        ddb = DDBSCAN('modules_config/clustering.txt').fit(X, sample_weight = sample_weight)
        t2 = time.perf_counter()
        
        # Black removed and is used for noise instead.
        unique_labels = set(ddb.labels_[:,0])

        # Number of polynomial clusters in labels, ignoring noise if present.
        n_superclusters = len(unique_labels) - (1 if -1 in ddb.labels_[:,0] else 0)

        for k in unique_labels:
            if k == -1:
                break # noise: the unclustered

            class_member_mask = (ddb.labels_[:,0] == k)
            xy = np.unique(X[class_member_mask],axis=0)
            x = xy[:, 0]; y = xy[:, 1]
            
            # both core and neighbor samples are saved in the cluster in the event
            if k>-1 and len(x)>1:
                cl = Cluster(xy,self.rebin,image_fr_vignetted,image_fr_zs_vignetted,self.options.geometry,debug=False)
                cl.iteration = 0
                cl.nclu = k
                cl.pearson = 999#p_value
                superclusters.append(cl)
                
        t2 = time.perf_counter()

        ## DEBUG MODE

        return superclusters
        
    def getTracks(self,plot=True):
        from skimage.transform import (hough_line, hough_line_peaks)
        # Classic straight-line Hough transform
        image = self.imagelog
        h, theta, d = hough_line(image)
        print("tracks found")
        
        tracks = []
        thr = 0.8 * np.amax(h)
        #######################   IMPLEMENT HERE THE SAVING OF THE TRACKS ############
        # loop over prominent tracks
        itrk = 0
        for _, angle, dist in zip(*hough_line_peaks(h, theta, d,threshold=thr)):
            print("Track # ",itrk)
            #points_along_trk = np.zeros((self.image.shape[1],self.image.shape[0]))
            points_along_trk = []
            for x in range(self.image.shape[1]):
                y = min(self.image.shape[0],max(0,int((dist - x * np.cos(angle)) / np.sin(angle))))
                #points_along_trk[x,y] = self.image[y,x]
                #print "adding point: %d,%d,%f" % (x,y,self.image[y,x])
                # add a halo fo +/- 20 pixels to calculate the lateral profile
                for iy in range(int(y)-5,int(y)+5):
                    if iy<0 or iy>=self.image.shape[0]: continue
                    points_along_trk.append((x,iy,self.image[iy,x]))
            xy = np.array(points_along_trk)
            trk = Cluster(xy,self.rebin)
            tracks.append(trk)
            itrk += 1
        ###################################
            
        if plot:
            # Generating figure
            from matplotlib import cm
            fig, ax = plt.subplots(2, 1, figsize=(18, 6))
            #ax = axes.ravel()

            ax[0].imshow(image, cmap=cm.gray)
            ax[0].set_title('Camera image')
            #ax[0].set_axis_off()            

            ax[1].imshow(image, cmap=cm.gray)
            for _, angle, dist in zip(*hough_line_peaks(h, theta, d,threshold=thr)):
                y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
                y1 = (dist - image.shape[1] * np.cos(angle)) / np.sin(angle)
                ax[1].plot((0, image.shape[1]), (y0, y1), '-r')
            ax[1].set_xlim((0, image.shape[1]))
            ax[1].set_ylim((image.shape[0], 0))
            #ax[1].set_axis_off()
            ax[1].set_title('Fitted tracks')

            plt.tight_layout()
            #plt.show()
            outname = self.options.plotDir
            if outname and not os.path.exists(outname):
                os.system("mkdir -p "+outname)
                os.system("cp ~/cernbox/www/Cygnus/index.php "+outname)
            for ext in ['pdf']:
                plt.savefig('{pdir}/{name}.{ext}'.format(pdir=outname,name=self.name,ext=ext))
            plt.gcf().clear()

        return tracks
        
    def plotClusterFullResolution(self,clusters):
        outname = self.options.plotDir
        for k,cl in enumerate(clusters):
            cl.plotFullResolution('{pdir}/{name}_cluster{iclu}'.format(pdir=outname,name=self.name,iclu=k))

    def calcProfiles(self,clusters,plot=False):
        for k,cl in enumerate(clusters):
            profName = '{name}_cluster{iclu}'.format(name=self.name,iclu=k)
            cl.calcProfiles(name=profName,plot=plot)
                             
    def plotProfiles(self,clusters):
        print ("plot profiles...")
        outname = self.options.plotDir
        canv = ROOT.TCanvas('c1','',1200,600)
        for k,cl in enumerate(clusters):
            for dir in ['long','lat']:
                profName = '{name}_cluster{iclu}_{dir}'.format(name=self.name,iclu=k,dir=dir)
                prof = cl.getProfile(dir)
                if prof and cl.widths[dir]>0.2: # plot the profiles only of sufficiently long snakes (>200 um)
                    prof.Draw("pe1")
                    for ext in ['pdf']:
                        canv.SaveAs('{pdir}/{name}profile.{ext}'.format(pdir=outname,name=profName,ext=ext))

class SnakesProducer:
    def __init__(self,sources,params,options,geometry):
        self.picture     = sources['picture']     if 'picture' in sources else None
        self.pictureHD   = sources['pictureHD']   if 'pictureHD' in sources else None
        self.picturezsHD = sources['picturezsHD'] if 'picturezsHD' in sources else None
        self.pictureOri  = sources['pictureOri']  if 'pictureOri' in sources else None
        self.vignette    = sources['vignette']    if 'vignette' in sources else None
        self.name        = sources['name']        if 'name' in sources else None
        self.algo        = sources['algo']        if 'algo' in sources else 'DBSCAN'
        
        self.snakeQualityLevel = params['snake_qual']   if 'snake_qual' in params else 3
        self.plot2D            = params['plot2D']       if 'plot2D' in params else False
        self.plotpy            = params['plotpy']       if 'plotpy' in params else False
        self.plotprofiles      = params['plotprofiles'] if 'plotprofiles' in params else False

        self.options = options
        self.geometry = geometry
        geometryPSet   = open('modules_config/geometry_{det}.txt'.format(det=options.geometry),'r')
        geometryParams = eval(geometryPSet.read())

        self.run_cosmic_killer = self.options.cosmic_killer
        if self.run_cosmic_killer:
            from clusterMatcher import ClusterMatcher
            # cosmic killer parameters
            cosmicKillerPars = open('modules_config/clusterMatcher.txt','r')
            killer_params = eval(cosmicKillerPars.read())
            killer_params.update(geometryParams)
            self.cosmic_killer = ClusterMatcher(killer_params)

        
    def run(self):
        ret = []
        if any([x==None for x in (self.picture.any(),self.pictureHD.any(),self.picturezsHD.any(),self.name)]):
            return ret

        t0 = time.perf_counter()
        
        # Cluster reconstruction on 2D picture
        snfac = SnakesFactory(self.picture,self.pictureHD,self.picturezsHD,self.pictureOri,self.vignette,self.name,self.options,self.geometry)

        # this plotting is only the pyplot representation.
        # Doesn't work on MacOS with multithreading for some reason... 
        if self.algo=='DBSCAN':
            snakes = snfac.getClusters(plot=self.plotpy)

            # supercluster energy calibration for the saturation effect
            fileCalPar = open('modules_config/energyCalibrator.txt','r')
            params = eval(fileCalPar.read())
            calibrator = EnergyCalibrator(params,self.options.debug_mode)
            
            for sclu in snakes:
                if self.options.calibrate_clusters:
                    calEnergy,slicesCalEnergy,centers = calibrator.calibratedEnergy(sclu.hits_fr)
                else:
                    calEnergy,slicesCalEnergy,centers = -1,[],[]
                if self.options.debug_mode:
                    print ( "SUPERCLUSTER BARE INTEGRAL = {integral:.1f}".format(integral=sclu.integral()) )
                sclu.calibratedEnergy = calEnergy
                sclu.nslices = len(slicesCalEnergy)
                sclu.energyprofile = slicesCalEnergy
                sclu.centers = centers
                sclu.pathlength = -1 if self.options.calibrate_clusters==False else calibrator.clusterLength()    
            
        elif self.algo=='HOUGH':
            clusters = []
            snakes = snfac.getTracks(plot=self.plotpy)            
        t1 = time.perf_counter()
        if self.options.debug_mode: print(f"FULL RECO in {t1 - t0:0.4f} seconds")
            
        # print "Get light profiles..."
        snfac.calcProfiles(snakes,plot=self.plotpy)
        t2 = time.perf_counter()
        if self.options.debug_mode: print(f"cluster shapes in {t2 - t1:0.4f} seconds")

        # run the cosmic killer: it makes sense only on superclusters
        if self.run_cosmic_killer:
            for ik,killerCand in enumerate(snakes):
                targets = [snakes[it] for it in range(len(snakes)) if it!=ik]
                self.cosmic_killer.matchClusters(killerCand,targets)
            t3 = time.perf_counter()
            if self.options.debug_mode: print(f"cosmic killer in {t3 - t2:0.4f} seconds")

        # snfac.calcProfiles(snakes) # this is for BTF
        
        # sort snakes by light integral
        snakes = sorted(snakes, key = lambda x: x.integral(), reverse=True)
        # and reject discharges (round)
        #snakes = [x for x in snakes if x.qualityLevel()>=self.snakeQualityLevel]
        
        # plotting
        if self.plot2D:       snfac.plotClusterFullResolution(snakes)
        if self.plotprofiles: snfac.plotProfiles(snakes)

        return snakes

/home/amaro/notebooks/reconstruction
Welcome to JupyROOT 6.22/09


/home/amaro/.local/lib/python3.10/site-packages/root_numpy/__init__.py:40: RuntimeWarning: numpy 1.22.2 is currently installed but you installed root_numpy against numpy 1.21.4. Please consider reinstalling root_numpy for this numpy version.
  warnings.warn(


## Main file

In [8]:
%tb

from multiprocessing import Pool,set_start_method,TimeoutError
from subprocess import Popen, PIPE
import signal
import matplotlib.pyplot as plt

import pdb

import os,math,sys,random
import numpy as np
import time

import ROOT
ROOT.gROOT.SetBatch(True)
from root_numpy import hist2array
from cameraChannel import cameraTools, cameraGeometry

from snakes import SnakesProducer
from output import OutputTree
from treeVars import AutoFillTreeProducer
import swiftlib as sw

from pathlib import Path

import pandas as pd

import warnings

warnings.filterwarnings("ignore")

# this kills also the still running subprocesses.
# use with a safe MAX TIMEOUT duration, since it will kill everything
def terminate_pool_2(pool):
    print ("Some subprocess timed out. Killing it brutally.")
    os.system('killall -9 python3.8')

# this still stucks
def terminate_pool(pool):
    print ("Some subprocess timed out. Killing it brutally.")
    for p in pool._pool:
        print ("KILLING PID ",p.pid)
        os.kill(p.pid, 9)
    pool.close()
    pool.terminate()
    pool.join()

def split(array, nrows, ncols):
    r, h = array.shape
    return (array.reshape(h//nrows, nrows, -1, ncols)
                 .swapaxes(1, 2)
                 .reshape(-1, nrows, ncols))
    
import utilities
utilities = utilities.utils()

class analysis:

    def __init__(self,options):
        self.rebin = options.rebin        
        self.options = options
        self.pedfile_fullres_name = options.pedfile_fullres_name
        self.tmpname = options.tmpname
        geometryPSet   = open('modules_config/geometry_{det}.txt'.format(det=options.geometry),'r')
        geometryParams = eval(geometryPSet.read())
        self.cg = cameraGeometry(geometryParams)
        self.xmax = self.cg.npixx

        if not os.path.exists(self.pedfile_fullres_name):
            print("WARNING: pedestal file with full resolution ",self.pedfile_fullres_name, " not existing. First calculate them...")
            self.calcPedestal(options,1)
        if not options.justPedestal:
           #print("Pulling pedestals...")
           # first the one for clustering with rebin
            ctools = cameraTools(self.cg)
           # then the full resolution one
            pedrf_fr = ROOT.TFile.Open(self.pedfile_fullres_name)
            self.pedmap_fr = pedrf_fr.Get('pedmap').Clone()
            self.pedmap_fr.SetDirectory(0)
            self.pedarr_fr = hist2array(self.pedmap_fr).T
            self.noisearr_fr = ctools.noisearray(self.pedmap_fr).T
            pedrf_fr.Close()
            if options.vignetteCorr:
                self.vignmap = ctools.loadVignettingMap()
            else:
                self.vignmap = np.ones((self.xmax, self.xmax))
             

    # the following is needed for multithreading
    def __call__(self,evrange=(-1,-1,-1)):
        if evrange[0]==-1:
            outfname = self.options.outFile
        else:
            outfname = '{base}_chunk{ij}.root'.format(base=self.options.outFile.split('.')[0],ij=evrange[0])
        self.beginJob(outfname)
        self.reconstruct(evrange)
        self.endJob()
        
    def beginJob(self,outfname):
        # prepare output file
        self.outputFile = ROOT.TFile.Open(outfname, "RECREATE")
        # prepare output tree
        self.outputTree = ROOT.TTree("Events","Tree containing reconstructed quantities")
        self.outTree = OutputTree(self.outputFile,self.outputTree)
        self.autotree = AutoFillTreeProducer(self.outTree)

        self.outTree.branch("run", "I")
        self.outTree.branch("event", "I")
        self.outTree.branch("pedestal_run", "I")
        if self.options.camera_mode:
            self.autotree.createCameraVariables()
            self.autotree.createClusterVariables('cl')
            self.autotree.createClusterVariables('sc')
        if self.options.pmt_mode:
            self.autotree.createPMTVariables()

    def endJob(self):
        self.outTree.write()
        self.outputFile.Close()
        
    def getNEvents(self):
        tf = sw.swift_read_root_file(self.tmpname) #tf = ROOT.TFile.Open(self.rfile)
        ret = int(len(tf.GetListOfKeys())/2) if (self.options.daq=='midas' and self.options.pmt_mode) else len(tf.GetListOfKeys())
        tf.Close()
        return ret

    def calcPedestal(self,options,alternativeRebin=-1):
        maxImages=options.maxEntries
        nx=ny=self.xmax
        rebin = self.rebin if alternativeRebin<0 else alternativeRebin
        nx=int(nx/rebin); ny=int(ny/rebin); 
        #pedfilename = 'pedestals/pedmap_ex%d_rebin%d.root' % (options.pedexposure,rebin)
        pedfilename = 'pedestals/pedmap_run%s_rebin%d.root' % (options.run,rebin)
        
        pedfile = ROOT.TFile.Open(pedfilename,'recreate')
        pedmap = ROOT.TH2D('pedmap','pedmap',nx,0,self.xmax,ny,0,self.xmax)
        pedmapS = ROOT.TH2D('pedmapsigma','pedmapsigma',nx,0,self.xmax,ny,0,self.xmax)

        pedsum = np.zeros((nx,ny))
        
        tf = sw.swift_read_root_file(self.tmpname)
        #tf = ROOT.TFile.Open(self.rfile)

        # first calculate the mean 
        numev = 0
        for i,e in enumerate(tf.GetListOfKeys()):
            iev = i if self.options.daq != 'midas' and self.options.pmt_mode else i/2 # when PMT is present
            if iev in self.options.excImages: continue
            if maxImages>-1 and i>min(len(tf.GetListOfKeys()),maxImages): break
            
            name=e.GetName()
            obj=e.ReadObj()

            if not obj.InheritsFrom('TH2'): continue
            print("Calc pedestal mean with event: ",name)
            if rebin>1:
                obj.RebinX(rebin);
                obj.RebinY(rebin); 
            arr = hist2array(obj)
            pedsum = np.add(pedsum,arr)
            numev += 1
        pedmean = pedsum / float(numev)

        # now compute the rms (two separate loops is faster than one, yes)
        numev=0
        pedsqdiff = np.zeros((nx,ny))
        for i,e in enumerate(tf.GetListOfKeys()):
            iev = i if self.options.daq != 'midas' and self.options.pmt_mode else i/2 # when PMT is present
            if iev in self.options.excImages: continue
            if maxImages>-1 and i>min(len(tf.GetListOfKeys()),maxImages): break
            
            name=e.GetName()
            obj=e.ReadObj()
            if not obj.InheritsFrom('TH2'): continue
            print("Calc pedestal rms with event: ",name)
            if rebin>1:
                obj.RebinX(rebin);
                obj.RebinY(rebin); 
            arr = hist2array(obj)
            pedsqdiff = np.add(pedsqdiff, np.square(np.add(arr,-1*pedmean)))
            numev += 1
        pedrms = np.sqrt(pedsqdiff/float(numev-1))

        # now save in a persistent ROOT object
        for ix in range(nx):
            for iy in range(ny):
                pedmap.SetBinContent(ix+1,iy+1,pedmean[ix,iy]);
                pedmap.SetBinError(ix+1,iy+1,pedrms[ix,iy]);
                pedmapS.SetBinContent(ix+1,iy+1,pedrms[ix,iy]);
        tf.Close()

        pedfile.cd()
        pedmap.Write()
        pedmapS.Write()
        pedmean1D = ROOT.TH1D('pedmean','pedestal mean',500,97,103)
        pedrms1D = ROOT.TH1D('pedrms','pedestal RMS',500,0,5)
        for ix in range(nx):
            for iy in range(ny):
               pedmean1D.Fill(pedmap.GetBinContent(ix,iy)) 
               pedrms1D.Fill(pedmap.GetBinError(ix,iy)) 
        pedmean1D.Write()
        pedrms1D.Write()
        pedfile.Close()
        print("Pedestal calculated and saved into ",pedfilename)


    def reconstruct(self,n=4,evrange=(-1,-1,-1)):

        ROOT.gROOT.Macro('rootlogon.C')
        ROOT.gStyle.SetOptStat(0)
        ROOT.gStyle.SetPalette(ROOT.kRainBow)
        savErrorLevel = ROOT.gErrorIgnoreLevel; ROOT.gErrorIgnoreLevel = ROOT.kWarning
        
        rebinned_list_mean = []
        incorrect  = 0
        incorrect2 = 0

        tf = sw.swift_read_root_file(self.tmpname)
        #tf = ROOT.TFile.Open(self.rfile)
        #c1 = ROOT.TCanvas('c1','',600,600)
        ctools = cameraTools(self.cg)
        #print("Reconstructing event range: ",evrange[1],"-",evrange[2])
        # loop over events (pictures)
        for iobj,key in enumerate(tf.GetListOfKeys()) :

            t1_start = time.perf_counter()
            iev = int(iobj/2) if self.options.daq == 'midas' and self.options.pmt_mode else iobj
            #print("max entries = ",self.options.maxEntries)
            if self.options.maxEntries>0 and iev==max(evrange[0],0)+self.options.maxEntries: break
            if sum(evrange[1:])>-2:
                if iev<evrange[1] or iev>evrange[2]: continue

            name=key.GetName()
            obj=key.ReadObj()

            # Routine to skip some images if needed
            if iev in self.options.excImages: continue

            if obj.InheritsFrom('TH2'):
                if self.options.daq == 'btf':
                    run,event=(int(name.split('_')[0].split('run')[-1].lstrip("0")),int(name.split('_')[-1].lstrip("0")))
                elif self.options.daq == 'h5':
                    run,event=(int(name.split('_')[0].split('run')[-1]),int(name.split('_')[-1]))
                else:
                    run,event=(int(name.split('_')[1].split('run')[-1].lstrip("0")),int(name.split('_')[-1].split('ev')[-1]))
                #print("Processing Run: ",run,"- Event ",event,"...")
                
                testspark=100*self.cg.npixx*self.cg.npixx+9000000
                if obj.Integral()>testspark:
                          print("Run ",run,"- Event ",event," has spark, will not be analyzed!")
                          continue
                            
                self.outTree.fillBranch("run",run)
                self.outTree.fillBranch("event",event)
                self.outTree.fillBranch("pedestal_run", int(self.options.pedrun))

            if self.options.camera_mode:
                if obj.InheritsFrom('TH2'):

                    # Upper Threshold full image
                    pic_fullres = obj.Clone(obj.GetName()+'_fr')
                    img_fr = hist2array(pic_fullres).T

                    # Upper Threshold full image
                    img_cimax = np.where(img_fr < self.options.cimax, img_fr, 0)
                    
                    # zs on full image + saturation correction on full image
                    if self.options.saturation_corr:
                    	#print("you are in saturation correction mode")
                    	img_fr_sub = ctools.pedsub(img_cimax,self.pedarr_fr)
                    	img_fr_satcor = ctools.satur_corr(img_fr_sub) 
                    	img_fr_zs  = ctools.zsfullres(img_fr_satcor,self.noisearr_fr,nsigma=self.options.nsigma)
                    	img_rb_zs  = ctools.arrrebin(img_fr_zs,self.rebin)
                        
                    # skip saturation and set satcor =img_fr_sub 
                    else:
                        #print("you are in poor mode")
                        img_fr_sub = ctools.pedsub(img_cimax,self.pedarr_fr)
                        img_fr_satcor = img_fr_sub  
                        img_fr_zs  = ctools.zsfullres(img_fr_satcor,self.noisearr_fr,nsigma=self.options.nsigma)
                        img_rb_zs  = ctools.arrrebin(img_fr_zs,self.rebin)
                    #print(np.asarray(img_fr_sub).shape)
                    #print(np.asarray(img_fr_satcor).shape)
                    #print(np.asarray(img_fr_zs).shape)
                    #print(np.asarray(img_rb_zs).shape)
                                        
                    # zs on full image + saturation correction on full image
                    #algo = 'DBSCAN'
                    #if self.options.type in ['beam','cosmics']: algo = 'HOUGH'
                    #snprod_inputs = {'picture': img_rb_zs, 'pictureHD': img_fr_satcor, 'picturezsHD': img_fr_zs, 'pictureOri': img_fr, 'vignette': self.vignmap, 'name': name, 'algo': algo}
                    #plotpy = self.options.jobs < 2 # for some reason on macOS this crashes in multicore
                    #snprod_params = {'snake_qual': 3, 'plot2D': False, 'plotpy': False, 'plotprofiles': False}
                    #snprod = SnakesProducer(snprod_inputs,snprod_params,self.options,self.cg)

                    #snakes = snprod.run()


                    #rebor = ctools.arrrebin(img_fr_zs,2048//n)
                    rebor = ctools.arrrebin(img_fr_zs,2048//n)
                    rebinned_list_mean.append(rebor)
                    #rebor2 = ctools.arrrebin(img_fr_satcor,512)
                 
                    #if (np.amax(rebor) < 0.461):
                    #    incorrect += 1
                    #if (np.amax(rebor2) < -0.006):
                    #    incorrect2 += 1
                    if (np.amax(rebor) < 0.5):
                        #if (iobj in [14, 292, 400, 696, 928]):
                        #    print("Iobj",iobj,"correctly removed by mean lower than 0.461")
                        #else:
                        #print("Iobj",iobj,"incorrectly removed by mean lower than 0.461")
                            #print("lost by mistake, max value is",np.amax(rebor))
                    #        print(len(snakes), "lost by mistake, max value is",np.amax(rebor))
                        incorrect += 1
                    else:
                        incorrect2 += 1
                        #print(incorrect, "selected incorectly so far")
                    #elif(len(snakes) == 0):
                    #    print("AAAH sem sinal e passou, mula" + str(iobj))
                    #if (np.amax(rebor2) < -0.006):
                    #    if (iobj in [14, 292, 400, 696, 928]):
                    #        print("Iobj",iobj,"correctly removed by mean lower than 0.005")
                    #    else:
                    #        print("Iobj",iobj,"incorrectly removed by mean lower than 0.005")
                            #print("lost by mistake, max value is",np.amax(rebor))
                    #        print(len(snakes), "lost by mistake, max value is",np.amax(rebor))
                    #        incorrect2 += 1
                    #        print(incorrect2, "selected incorectly so far")
                    #elif(len(snakes) == 0):
                    #    print("AAAH sem sinal e passou, mula iobj:" + str(iobj))



        ROOT.gErrorIgnoreLevel = savErrorLevel

        print("Total Incorrect="+str(incorrect)+" Correct="+str(incorrect2))
        return rebinned_list_mean

if __name__ == '__main__':

    for entry in path.iterdir():
        print(entry.name)

        from optparse import OptionParser

        parser = OptionParser(usage='%prog h5file1,...,h5fileN [opts] ')
        parser.add_option('-r', '--run', dest='run', default='02163', type='string', help='run number with 5 characteres')
        parser.add_option('-j', '--jobs', dest='jobs', default=1, type='int', help='Jobs to be run in parallel (-1 uses all the cores available)')
        parser.add_option(      '--max-entries', dest='maxEntries', default=numberofevents[0], type='float', help='Process only the first n entries')
        parser.add_option(      '--pdir', dest='plotDir', default='./', type='string', help='Directory where to put the plots')
        parser.add_option(      '--tmp',  dest='tmpdir', default=None, type='string', help='Directory where to put the input file. If none is given, /tmp/<user> is used')
        parser.add_option(      '--max-hours', dest='maxHours', default=400, type='float', help='Kill a subprocess if hanging for more than given number of hours.')
        parser.add_option('-f', '--frun', dest='yadda', default='02163', type='string', help='run number with 5 characteres')

        (options, args) = parser.parse_args()

        f = open("./configFilef55.txt", "r")
        params = eval(f.read())

        for k,v in params.items():
            setattr(options,k,v)

        run = int(options.run)
        options.debug_mode = 0
        if options.debug_mode == 1:
            setattr(options,'outFile','reco_run%d_%s_debug.root' % (run, options.tip))
            if options.ev: options.maxEntries = options.ev + 1
            #if options.daq == 'midas': options.ev +=0.5 
        else:
            setattr(options,'outFile','reco_run%05d_%s.root' % (run, options.tip))

        if not hasattr(options,"pedrun"):
            pf = open("pedestals/pedruns.txt","r")
            peddic = eval(pf.read())
            options.pedrun = -1
            for runrange,ped in peddic.items():
                if int(runrange[0])<=run<=int(runrange[1]):
                    options.pedrun = int(ped)
                    #print("Will use pedestal run %05d, valid for run range [%05d - %05d]" % (int(ped), int(runrange[0]), (runrange[1])))
                    break
            assert options.pedrun>0, ("Didn't find the pedestal corresponding to run ",run," in the pedestals/pedruns.txt. Check the dictionary inside it!")

        setattr(options,'pedfile_fullres_name', 'pedestals/pedmap_run%s_rebin1.root' % (options.pedrun))

        #inputf = inputFile(options.run, options.dir, options.daq)

        os.environ['USER'] = 'amaro'
        os.environ['ROOTSYS'] = '/content/root_build'
        USER = os.environ['USER']
        tmpdir = '/mnt/ssdcache/' if os.path.exists('/mnt/ssdcache/') else '/content/'
        # override the default, if given by option
        options.tmpname = str(entry)

        if options.justPedestal:
            ana = analysis(options)
            print("Pedestals done. Exiting.")
            if options.donotremove == False:
                sw.swift_rm_root_file(options.tmpname)
            sys.exit(0)     

        ana = analysis(options)
        nev = ana.getNEvents() if options.maxEntries == -1 else int(options.maxEntries)
        print("This run has ",nev," events.")
        #print("Will save plots to ",options.plotDir)
        os.system('cp utils/index.php {od}'.format(od=options.plotDir))

        nThreads = 1
        if options.jobs==-1:
            import multiprocessing
            nThreads = multiprocessing.cpu_count()
        else:
            nThreads = 1

        if nThreads>1:
            print ("RUNNING USING ",nThreads," THREADS.")
            nj = int(nev/nThreads)
            chunks = [(ichunk,i,min(i+nj-1,nev)) for ichunk,i in enumerate(range(0,nev,nj))]
            print(chunks)
            pool = Pool(nThreads)
            ret = list(pool.apply_async(ana,args=(c, )) for c in chunks)
            try:
                if options.maxHours>0:
                    maxTime = options.maxHours * 3600
                else:
                    # 64-bit integer, converted from nanoseconds to seconds, and subtracting 0.1 just to be in bounds.
                    maxTime = 2 ** 63 / 1e9 - 0.1
                print([r.get(timeout=maxTime) for r in ret])
                pool.close()
                pool.terminate()
                pool.join()
            except TimeoutError:
                print("except")
                terminate_pool_2(pool)
            print("Now hadding the chunks...")
            base = options.outFile.split('.')[0]
            os.system('{rootsys}/bin/hadd -k -f {base}.root {base}_chunk*.root'.format(rootsys=os.environ['ROOTSYS'],base=base))
            os.system('rm {base}_chunk*.root'.format(base=base))
        else:
            for n in sizes:
                ana.beginJob(options.outFile)
                rebinned_list_mean = ana.reconstruct(n)
                ana.endJob()
                np.savetxt("../moved_means/%s_%d.txt"%(entry.name,n),np.array(rebinned_list_mean).reshape(-1,n**2))

        #### FOR SOME REASON THIS DOESN'T WORK IN BATCH.
        # now add the git commit hash to track the version in the ROOT file
        # tf = ROOT.TFile.Open(options.outFile,'update')
        # githash = ROOT.TNamed("gitHash",str(utilities.get_git_revision_hash()).replace('\n',''))
        # githash.Write()
        # tf.Close()

        if options.donotremove == False:
            sw.swift_rm_root_file(options.tmpname)

No traceback available to show.


histograms_Run08007.root
This run has  1000  events.
Total Incorrect=95 Correct=0
Total Incorrect=95 Correct=0
Total Incorrect=2 Correct=93
Total Incorrect=0 Correct=95
Total Incorrect=0 Correct=95
histograms_Run08009.root
This run has  1000  events.
Total Incorrect=95 Correct=0
Total Incorrect=95 Correct=0
Total Incorrect=5 Correct=90
Total Incorrect=0 Correct=95
Total Incorrect=0 Correct=95
histograms_Run08005.root
This run has  1000  events.
Total Incorrect=93 Correct=0
Total Incorrect=93 Correct=0
Total Incorrect=4 Correct=89
Total Incorrect=0 Correct=93
Total Incorrect=0 Correct=93
histograms_Run02163-005.root
This run has  1000  events.
Total Incorrect=411 Correct=86
Total Incorrect=190 Correct=307
Total Incorrect=10 Correct=487
Total Incorrect=0 Correct=497
Total Incorrect=0 Correct=497
histograms_Run08011.root
This run has  1000  events.
Total Incorrect=95 Correct=0
Total Incorrect=95 Correct=0
Total Incorrect=0 Correct=95
Total Incorrect=0 Correct=95
Total Incorrect=0 Correct=

In [9]:
#np.savetxt("median.txt",np.array(rebinned_list_median))

In [10]:
#np.savetxt("mean.txt",np.array(rebinned_list_mean).reshape(-1,(n//2)))

# TODO

+ OK Fazer os trem em boxplot
+ ver o tempo acima de 8seg
+ ver parametro pra fazer seleção de imagem
+ check tail by images with many points

# TOTALK

Cortar evento com energia demais?